In [93]:
import re
import numpy as np
import math
import matplotlib.pyplot as plt

mu_sol = 1.32712440018e11  # km^3/s^2

In [94]:
def read_parameters(date, filename='cassini_all_orbit.txt'):
    """
    Lê o arquivo de dados e retorna todos os parâmetros para uma data específica
    
    Args:
        date (str): Data no formato 'ano-Mes-dia'
        filename (str): Nome do arquivo de dados
    
    Returns:
        dict: Dicionário com todos os parâmetros orbitais
    """
    
    with open(filename, 'r') as file:
        content = file.read()
    
    pattern = rf'(\d+\.\d+) = A\.D\. {date}.*?(?=\n\d+\.\d+ = A\.D\.|\Z)'
    match = re.search(pattern, content, re.DOTALL)
    
    if not match:
        raise ValueError(f"Data {date} não encontrada no arquivo")
    
    block = match.group(0)
    
    params = {}
    
    jd_match = re.search(r'(\d+\.\d+) = A\.D\.', block)
    if jd_match:
        params['JDTDB'] = float(jd_match.group(1))
    
    param_pattern = r'([A-Za-z]+)\s*=\s*([-+]?\s*\d*\.?\d+(?:[Ee][-+]?\d+)?)'
    matches = re.findall(param_pattern, block)
    
    for key, value in matches:
        value_clean = re.sub(r'\s+', '', value)
        params[key] = float(value_clean)
    
    return params


In [95]:
def print_parameters(date, filename='cassini_all_orbit.txt'):
    """
    Lê e imprime todos os parâmetros para uma data específica
    """
    parametros = read_parameters(date, filename)
    
    print(f"Parâmetros orbitais para {date}:")
    print("=" * 50)
    
    # Lista de todos os parâmetros na ordem desejada
    parametros_info = [
        ('JDTDB', 'Julian Day Number, Barycentric Dynamical Time'),
        ('EC', 'Eccentricity, e'),
        ('QR', 'Periapsis distance, q (km)'),
        ('IN', 'Inclination w.r.t X-Y plane, i (degrees)'),
        ('OM', 'Longitude of Ascending Node, OMEGA, (degrees)'),
        ('W', 'Argument of Perifocus, w (degrees)'),
        ('Tp', 'Time of periapsis (Julian Day Number)'),
        ('N', 'Mean motion, n (degrees/sec)'),
        ('MA', 'Mean anomaly, M (degrees)'),
        ('TA', 'True anomaly, nu (degrees)'),
        ('A', 'Semi-major axis, a (km)'),
        ('AD', 'Apoapsis distance (km)'),
        ('PR', 'Sidereal orbit period (sec)')
    ]
    
    for key, description in parametros_info:
        if key in parametros:
            if key == 'JDTDB':
                print(f"{key:4} = {parametros[key]:.6f} # {description}")
            else:
                print(f"{key:4} = {parametros[key]:.15e} # {description}")
        else:
            print(f"{key:4} = NÃO ENCONTRADO # {description}")

In [96]:
def excentricity(r1, theta1, r2, theta2):
    return abs((r2 - r1) / (r1*np.cos(theta1) - r2*np.cos(theta2)))

def semiaxis(r, e, theta):
    return r * (1 + e * np.cos(theta)) / (1 - e**2)



def angular_momentum(r, theta, e, mu):
    """
    Calcula o momento angular de forma robusta para órbitas elípticas e hiperbólicas
    """
    try:
        if e < 1:
            # Órbita elíptica
            return np.sqrt(mu * r * (1 + e * np.cos(theta)))
        else:
            # Órbita hiperbólica
            p = r * (1 + e * np.cos(theta))  # parâmetro da órbita
            print(p)
            return np.sqrt(mu * p)
    except Exception as ex:
        print(f"Erro no cálculo do momento angular: {ex}")
        return np.nan

def tangencial_velocity(mu, h, e, theta):
    return mu / h * (1 + e * np.cos(theta))

def radial_velocity(mu, h, e, theta):
    return mu / h * e * np.sin(theta)

def velocity_magnitude(mu, h, e, theta):
    return mu / h * np.sqrt(e**2 + 2*e*np.cos(theta) + 1)

In [97]:
def eccentric_anomaly_from_mean_anomaly(M,e):
    error = 1
    E = M
    while error > 1e-10:
        E_new = E - (E - e * math.sin(E) - M) / (1 - e * math.cos(E))
        error = abs((E_new - E)/E_new)
        E = E_new
    return E

def eccentric_anomaly_from_true_anomaly(theta, e):
    """
    Versão robusta para calcular anomalia excêntrica
    """
    # Garantir que theta está entre 0 e 2π
    theta = theta % (2 * np.pi)
    
    # Para órbitas circulares
    if abs(e) < 1e-10:
        return theta
    
    # Para órbitas elípticas (e < 1)
    if e < 1:
        # Fórmula robusta usando atan2
        sin_theta = np.sin(theta)
        cos_theta = np.cos(theta)
        
        denominator = 1 + e * cos_theta
        if abs(denominator) < 1e-12:
            return theta
        
        sqrt_value = max(0, 1 - e**2)  # Evitar valores negativos por erro numérico
        sin_E = np.sqrt(sqrt_value) * sin_theta / denominator
        cos_E = (e + cos_theta) / denominator
        
        E = np.arctan2(sin_E, cos_E)
        if E < 0:
            E += 2 * np.pi
        return E
    
    # Para órbitas hiperbólicas (e > 1) - necessário para algumas órbitas
    else:
        # Usar fórmula hiperbólica
        sin_theta = np.sin(theta)
        cos_theta = np.cos(theta)
        
        denominator = 1 + e * cos_theta
        if abs(denominator) < 1e-12:
            return theta
        
        H = 2 * np.arctanh(np.sqrt((e - 1) / (e + 1)) * np.tan(theta / 2))
        return H

def mean_anomaly_from_eccentric_anomaly(E, e):
    """Calcula a anomalia média M a partir da anomalia excêntrica E"""
    if e < 1:
        return E - e * np.sin(E)
    else:
        # Para órbitas hiperbólicas
        return e * np.sinh(E) - E

def orbital_period(a, mu):
    return 2 * math.pi * math.sqrt(a**3 / mu)

def time_from_periapsis(theta, e, a, mu):
    """
    Calcula o tempo desde a passagem pelo periastro para uma dada anomalia verdadeira
    Retorna tempo em segundos
    """
    try:
        E = eccentric_anomaly_from_true_anomaly(theta, e)
        M = mean_anomaly_from_eccentric_anomaly(E, e)
        
        if e < 1:
            n = np.sqrt(mu / abs(a)**3)  # Movimento médio para elípticas
        else:
            # Para órbitas hiperbólicas, usar a definição apropriada
            n = np.sqrt(mu / abs(a)**3)
        
        return M / n
    except Exception as ex:
        print(f"Erro em time_from_periapsis: {ex}")
        return 0

def calculate_travel_time(theta1, theta2, e, a, mu):
    """
    Calcula o tempo para viajar de theta1 a theta2
    Retorna tempo em dias
    """
    try:
        t1 = time_from_periapsis(theta1, e, a, mu)
        t2 = time_from_periapsis(theta2, e, a, mu)
        
        # Determinar a direção do movimento
        delta_t = t2 - t1
        
        # Se delta_t é negativo e é órbita elíptica, a nave passou pelo periastro
        if delta_t < 0 and e < 1:
            T = orbital_period(a, mu)
            delta_t += T
        
        return delta_t / 86400  # Converter para dias
    except Exception as e:
        print(f"Erro no cálculo do tempo: {e}")
        return 0

In [98]:
def plot_orbit(e, a, theta1, theta2, r1, r2):
    """
    Ta bem ruim o plot

    Plota a órbita elíptica com a parte percorrida em linha contínua
    
    Args:
        e: excentricidade
        a: semi-eixo maior (km)
        theta1, theta2: anomalias verdadeiras inicial e final (graus)
        r1, r2: raios orbital inicial e final (km)
    """
    
    # Parâmetros da elipse
    b = a * np.sqrt(1 - e**2)  # semi-eixo menor
    c = e * a  # distância do centro ao foco
    
    # Gera pontos para a órbita completa
    theta_full = np.linspace(0, 2*np.pi, 1000)
    r_full = a * (1 - e**2) / (1 + e * np.cos(theta_full))
    
    # Converte para coordenadas cartesianas
    x_full = r_full * np.cos(theta_full)
    y_full = r_full * np.sin(theta_full)
    
    # Gera pontos para a parte percorrida
    if theta2 > theta1:
        theta_traveled = np.linspace(theta1, theta2, 200)
    else:
        # Se theta2 < theta1, assumimos que passa pelo periastro
        theta_traveled = np.concatenate([
            np.linspace(theta1, 2*np.pi, 100),
            np.linspace(0, theta2, 100)
        ])
    
    r_traveled = a * (1 - e**2) / (1 + e * np.cos(theta_traveled))
    x_traveled = r_traveled * np.cos(theta_traveled)
    y_traveled = r_traveled * np.sin(theta_traveled)
    
    # Cria a figura
    plt.figure(figsize=(10, 8))
    
    # Plot da órbita completa (tracejada)
    plt.plot(x_full, y_full, '--', color='gray', alpha=0.7, label='Órbita completa')
    
    # Plot da parte percorrida (contínua)
    plt.plot(x_traveled, y_traveled, '-', color='blue', linewidth=2, label='Trajetória percorrida')
    
    # Marca as posições inicial e final
    plt.plot(r1 * np.cos(theta1), r1 * np.sin(theta1), 'go', 
             markersize=8, label=f'Início (θ={theta1:.1f}°)')
    plt.plot(r2 * np.cos(theta2), r2 * np.sin(theta2), 'ro', 
             markersize=8, label=f'Fim (θ={theta2:.1f}°)')
    
    # Marca o Sol no foco
    plt.plot(0, 0, 'yo', markersize=15, label='Sol')
    plt.plot(0, 0, 'y*', markersize=20)
    
    # Marca o periastro e apoastro
    periastro = a * (1 - e)
    apoastro = a * (1 + e)
    
    plt.plot([-c, -c], [0, 0], 'kx', markersize=8, label='Centro')
    plt.plot([-c + a, -c - a], [0, 0], 'k--', alpha=0.3)  # Linha do semi-eixo maior
    
    # Configurações do gráfico
    plt.axis('equal')
    plt.grid(True, alpha=0.3)
    plt.xlabel('x (km)')
    plt.ylabel('y (km)')
    plt.title(f'Órbita Cassini - {date1} a {date2}\n'
              f'e = {e:.4f}, a = {a/1e6:.1f}×10⁶ km')
    plt.legend()
    
    # Adiciona informações textuais
    textstr = f'Δθ = {theta2 - theta1:.1f}°\n' \
              f'r₁ = {r1/1e6:.1f}×10⁶ km\n' \
              f'r₂ = {r2/1e6:.1f}×10⁶ km\n' \
              f'Periastro = {periastro/1e6:.1f}×10⁶ km\n' \
              f'Apoastro = {apoastro/1e6:.1f}×10⁶ km'
    
    plt.annotate(textstr, xy=(0.02, 0.98), xycoords='axes fraction',
                 verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Ajusta os limites para melhor visualização
    max_range = max(apoastro, np.max(np.abs(x_full)), np.max(np.abs(y_full)))
    # plt.xlim(-max_range*1.1, max_range*1.1)
    # plt.ylim(-max_range*1.1, max_range*1.1)
    
    plt.tight_layout()
    plt.show()

In [99]:
def dados_orbita(start, finish, printa_dados=False, plota_orbita=False):
    
    data1 = read_parameters(start)
    data2 = read_parameters(finish)

    theta1 = np.radians(data1['TA'])
    theta2 = np.radians(data2['TA'])
    r1 = data1['A'] * (1 - data1['EC']**2) / (1 + data1['EC'] * np.cos(theta1))
    r2 = data2['A'] * (1 - data2['EC']**2) / (1 + data2['EC'] * np.cos(theta2))

    e = excentricity(r1, theta1, r2, theta2)
    a = semiaxis(r1, e, theta1)
    h = angular_momentum(r1, theta1, e, mu_sol)
    vr1 = radial_velocity(mu_sol, h, e, theta1)
    vt1 = tangencial_velocity(mu_sol, h, e, theta1)
    v1 = velocity_magnitude(mu_sol, h, e, theta1)
    vr2 = radial_velocity(mu_sol, h, e, theta2)
    vt2 = tangencial_velocity(mu_sol, h, e, theta2)
    v2 = velocity_magnitude(mu_sol, h, e, theta2)
    real_time = (data2['JDTDB'] - data1['JDTDB']) # em dias
    theoretical_time = calculate_travel_time(theta1, theta2, e, a, mu_sol)
    erro_percentual = abs((theoretical_time - real_time) / real_time) * 100

    if printa_dados:
        print()
        print(f"Parâmetros calculados entre {start} e {finish}:")
        print("=" * 50)
        print(f"Excentricidade: {e:.15e}")
        print(f"Semi-eixo maior:{a:.15e} km ")
        print(f"Momento angular: {h:.15e} km^2/s ")
        print(f"Velocidade inicial {start}: vr = {vr1:.15e} km/s, vt = {vt1:.15e} km/s, v_total = {v1:.15e} km/s")
        print(f"Velocidade final {finish}: vr = {vr2:.15e} km/s, vt = {vt2:.15e} km/s, v_total = {v2:.15e} km/s")
        print(f"Tempo real de viagem: {real_time:.6f} dias")
        print(f"Tempo teórico de viagem: {theoretical_time:.6f} dias")
        print(f"Erro percentual: {erro_percentual:.6f} %")
        print("=" * 50)
        print()

    if plota_orbita:
        plot_orbit(e, a, theta1, theta2, r1, r2)

    return {
        'e': e,
        'a': a,
        'h': h,
        'vr_start': vr1,
        'vt_start': vt1,
        'v_start': v1,
        'vr_end': vr2,
        'vt_end': vt2,
        'v_end': v2,
        'real_time_days': real_time,
        'theoretical_time_days': theoretical_time,
        'error_percent': erro_percentual
    }

In [100]:
# orbita1 = dados_orbita('1997-Oct-16', '1998-Apr-26', printa_dados=True, plota_orbita=False)

In [ ]:
# orbita2 = dados_orbita('1998-Apr-26', '1998-Dec-03', printa_dados=True, plota_orbita=False)

In [ ]:
# orbita3 = dados_orbita('1998-Dec-03', '1999-Jun-24', printa_dados=True, plota_orbita=False)

In [ ]:
# orbita4 = dados_orbita('1999-Jun-24', '1999-Aug-18', printa_dados=True, plota_orbita=False)

In [ ]:
# orbita5 = dados_orbita('1999-Aug-18', '2000-Dec-30', printa_dados=True, plota_orbita=False)

In [105]:
orbita6 = dados_orbita('2000-Dec-30', '2004-Jul-01', printa_dados=True, plota_orbita=False)

-812725674.7011473

Parâmetros calculados entre 2000-Dec-30 e 2004-Jul-01:
Excentricidade: 2.545462744437581e+00
Semi-eixo maior:1.483243703084327e+08 km 
Momento angular: nan km^2/s 
Velocidade inicial 2000-Dec-30: vr = nan km/s, vt = nan km/s, v_total = nan km/s
Velocidade final 2004-Jul-01: vr = nan km/s, vt = nan km/s, v_total = nan km/s
Tempo real de viagem: 1279.000000 dias
Tempo teórico de viagem: nan dias
Erro percentual: nan %



C:\Users\JoaoBento\AppData\Local\Temp\ipykernel_8436\3481871726.py:21: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(mu * p)
C:\Users\JoaoBento\AppData\Local\Temp\ipykernel_8436\943137282.py:50: RuntimeWarning: invalid value encountered in arctanh
  H = 2 * np.arctanh(np.sqrt((e - 1) / (e + 1)) * np.tan(theta / 2))
